In [1]:
import pandas as pd
import numpy as np
import random
import os
import re
import subprocess
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
from xgboost import XGBClassifier

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [37]:
file

,content,sentiment
0,Om het nog bondiger te zeggen: ik stemde ooit ...,Negative
1,Volgens de peilingen stond PVV 2 weken geleden...,Negative
2,PVV stemmers mogen dan het geheugen hebben van...,Negative
3,"PVV wil geen vluchtelingen, Omtzigt wil geen e...",Neither
4,Dat weten we niet. PVV heeft veel strategische...,Neither
...,...,...
404,Ik heb vvd gestemd omdat ik een coalitie over ...,Negative
405,"Exact, wordt niet bien van het krakeel en het ...",Neither
406,De hysterische gedoe van nu kan wel later voor...,Negative
407,Als je het enkel hebt over de persoon van der ...,Positive


In [3]:
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text


In [4]:
file['content'] = file['content'].apply(remove_urls) 

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_898/3299540989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)


In [48]:
file

,content,sentiment
0,Om het nog bondiger te zeggen: ik stemde ooit ...,Negative
1,Volgens de peilingen stond PVV 2 weken geleden...,Negative
2,PVV stemmers mogen dan het geheugen hebben van...,Negative
3,"PVV wil geen vluchtelingen, Omtzigt wil geen e...",Neither
4,Dat weten we niet. PVV heeft veel strategische...,Neither
...,...,...
404,Ik heb vvd gestemd omdat ik een coalitie over ...,Negative
405,"Exact, wordt niet bien van het krakeel en het ...",Neither
406,De hysterische gedoe van nu kan wel later voor...,Negative
407,Als je het enkel hebt over de persoon van der ...,Positive


In [5]:

file['content'] = file['content'].apply(clean_text) 


/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_898/964840669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)


In [50]:
file

,content,sentiment
0,om het nog bondiger te zeggen ik stemde ooit l...,Negative
1,volgens de peilingen stond pvv weken geleden ...,Negative
2,pvv stemmers mogen dan het geheugen hebben van...,Negative
3,pvv wil geen vluchtelingen omtzigt wil geen ex...,Neither
4,dat weten we niet pvv heeft veel strategische ...,Neither
...,...,...
404,ik heb vvd gestemd omdat ik een coalitie over ...,Negative
405,exact wordt niet bien van het krakeel en het m...,Neither
406,de hysterische gedoe van nu kan wel later voor...,Negative
407,als je het enkel hebt over de persoon van der ...,Positive


In [4]:
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5782/3999237305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})


In [52]:
X

0      om het nog bondiger te zeggen ik stemde ooit l...
1      volgens de peilingen stond pvv  weken geleden ...
2      pvv stemmers mogen dan het geheugen hebben van...
3      pvv wil geen vluchtelingen omtzigt wil geen ex...
4      dat weten we niet pvv heeft veel strategische ...
                             ...                        
404    ik heb vvd gestemd omdat ik een coalitie over ...
405    exact wordt niet bien van het krakeel en het m...
406    de hysterische gedoe van nu kan wel later voor...
407    als je het enkel hebt over de persoon van der ...
408    zoek het hoefijzermodel eens op dan zie je dat...
Name: content, Length: 409, dtype: object

In [5]:
nlp = spacy.load("nl_core_news_sm")
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

#load word2vec embeddings
model_path = '/Users/rachael/Downloads/39/model.bin'
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)
vector_size = word_vectors.vector_size #100
#convert sentences feature to the average of word vectors
def sentence_to_avg_vec(tokens, model, vector_size):
    vec = np.zeros(vector_size)
    count = 0
    for token in tokens:
        if token in model:
            vec += model[token]
            count += 1
    if count != 0:
        vec /= count
    return vec
    

In [64]:
vector_size

100

In [62]:
def sentence_to_avg_vec(tokens, word_vectors, vector_size):
    return np.mean([word_vectors.get(token, np.zeros(vector_size)) for token in tokens], axis=0)

In [8]:
#define parameter range 
param_dist = {
    'learning_rate': [item/100 for item in range(1, 31)],
    'max_depth': [item for item in range(3, 11)],
    'subsample': [item/10 for item in range(5, 11)],
    'colsample_bytree': [item/10 for item in range(5, 11)],
    'lambda': [item/10 for item in range(0, 11)],
    'alpha': [item/10 for item in range(0, 11)],
}

best_score = 0
best_params = {}
test_scores = []
best_params_list = []
f1_scores = []
balanced_accuracies = []

In [55]:
X

0      om het nog bondiger te zeggen ik stemde ooit l...
1      volgens de peilingen stond pvv  weken geleden ...
2      pvv stemmers mogen dan het geheugen hebben van...
3      pvv wil geen vluchtelingen omtzigt wil geen ex...
4      dat weten we niet pvv heeft veel strategische ...
                             ...                        
404    ik heb vvd gestemd omdat ik een coalitie over ...
405    exact wordt niet bien van het krakeel en het m...
406    de hysterische gedoe van nu kan wel later voor...
407    als je het enkel hebt over de persoon van der ...
408    zoek het hoefijzermodel eens op dan zie je dat...
Name: content, Length: 409, dtype: object

In [9]:

# 分层K折交叉验证
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 循环每个折叠
for train_index, test_index in outer_cv.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]

    # 确保传递字符串数据
    trainX = trainX.astype(str)
    testX = testX.astype(str)

    # 进行tokenize
    X_train_tokenized = trainX.apply(tokenize)
    X_test_tokenized = testX.apply(tokenize)

     #map word embedding to tokenized train and test data
    X_train_embeddings = np.array([sentence_to_avg_vec(tokens, word_vectors, vector_size) for tokens in X_train_tokenized])
    X_test_embeddings= np.array([sentence_to_avg_vec(tokens, word_vectors, vector_size) for tokens in X_test_tokenized])

    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    randomized_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=100,  # 采样次数
        scoring='f1_macro',
        n_jobs=-1,  # 使用所有可用的核心
        cv=inner_cv,
        random_state=42,
        verbose=1  # 设置详细程度
    )

    # 拟合随机搜索
    randomized_search.fit(X_train_embeddings, trainy)

    # 获取最佳模型
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # 在测试集上进行预测
    test_predictions = best_model.predict(X_test_embeddings)
    balanced_acc = f1_score(testy, test_predictions,average='macro')

    # 存储结果
    balanced_accuracies.append(balanced_acc)
    best_params_list.append(best_params)

    print(f"Fold Balanced Accuracy: {balanced_acc}")
    print(f"Best parameters: {best_params}")

# 汇总结果

mean_balanced_acc = np.mean(balanced_accuracies)
std_balanced_acc = np.std(balanced_accuracies)

print(f"Mean Balanced Accuracy: {mean_balanced_acc}")
print(f"Standard deviation of Balanced Accuracies: {std_balanced_acc}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold Balanced Accuracy: 0.27336860670194
Best parameters: {'subsample': 0.9, 'max_depth': 8, 'learning_rate': 0.29, 'lambda': 0.2, 'colsample_bytree': 1.0, 'alpha': 0.2}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold Balanced Accuracy: 0.4132505175983437
Best parameters: {'subsample': 1.0, 'max_depth': 10, 'learning_rate': 0.3, 'lambda': 0.0, 'colsample_bytree': 1.0, 'alpha': 0.5}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold Balanced Accuracy: 0.3942077934013417
Best parameters: {'subsample': 0.5, 'max_depth': 9, 'learning_rate': 0.26, 'lambda': 0.6, 'colsample_bytree': 0.8, 'alpha': 0.9}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold Balanced Accuracy: 0.4579365079365079
Best parameters: {'subsample': 0.6, 'max_depth': 10, 'learning_rate': 0.25, 'lambda': 0.3, 'colsample_bytree': 0.8, 'alpha': 0.7}
Fitting 3 folds for each of 100 candidates, totalling 300 fit

In [10]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'alpha': 0.2, 'colsample_bytree': 1.0, 'lambda': 0.2, 'learning_rate': 0.29, 'max_depth': 8, 'subsample': 0.9}


In [6]:
most_common_params = {'subsample': 0.6, 'max_depth': 10, 'learning_rate': 0.25, 'lambda': 0.3, 'colsample_bytree': 0.8, 'alpha': 0.7}


In [7]:
X_sentences = [item for item in list(X)]
X_tokenized = X.apply(tokenize)
X_embeddings = np.array([sentence_to_avg_vec(tokens, word_vectors, 100) for tokens in X_tokenized])

final_model = XGBClassifier(use_label_encoder=False, random_state=42,**most_common_params)
final_model.fit(X_embeddings, y)

XGBClassifier(alpha=0.7, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.3, learning_rate=0.25,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, ...)

In [8]:
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
#the data verified by the third person will be used as the train set finally
test = test[['content','sentiment']]
test['content'] = test['content'].apply(remove_urls) 
test['content'] = test['content'].apply(clean_text) 
test['sentiment'] = test['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
test_tokenized = test['content'].apply(tokenize)
test_embedding = np.array([sentence_to_avg_vec(tokens,word_vectors,100) for tokens in test_tokenized])
test_predictions = final_model.predict(test_embedding)
test_score = accuracy_score(test['sentiment'], test_predictions)

In [9]:
conf_matrix = confusion_matrix(test['sentiment'], test_predictions)
class_report = classification_report(test['sentiment'], test_predictions)

In [10]:
print(class_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.63      0.95      0.76       245
           2       0.70      0.21      0.32       124

    accuracy                           0.64       407
   macro avg       0.45      0.39      0.36       407
weighted avg       0.60      0.64      0.56       407

